In [1]:
import json
import pandas as pd
df = pd.read_json(r'dataset\response_1692132599095.json', dtype='unicode')

In [2]:
df

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,29188.0,568028572421,1,613030688168.0,8708037839.0,29430.0,...,21000000.0,21000000.0,69045.0,-57.72593,2021-11-10T14:24:11.849Z,67.81,42944.48019,2013-07-06T00:00:00.000Z,None,2023-08-15T20:49:12.343Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1828.72,219831455559,2,219831455559.0,4367028090.0,1846.04,...,120142222.589507,nan,4878.26,-62.49158,2021-11-10T14:24:19.604Z,0.432979,422497.63325,2015-10-20T00:00:00.000Z,"{'times': 82.77406462572597, 'currency': 'btc'...",2023-08-15T20:49:19.160Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.9988579999999999,83330198104,3,83330198104.0,16973712754.0,1.001,...,83410866925.5156,nan,1.32,-24.49275,2018-07-24T00:00:00.000Z,0.572521,74.49716,2015-03-02T00:00:00.000Z,None,2023-08-15T20:45:00.441Z
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,237.46,36454739107,4,47388081798.0,428728230.0,240.49,...,153856150.0,200000000.0,686.31,-65.47601,2021-05-10T07:24:17.097Z,0.0398177,594963.06423,2017-10-19T00:00:00.000Z,None,2023-08-15T20:49:10.303Z
4,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,0.605296,31993608622,5,60608551761.0,1170330739.0,0.635508,...,99988519823.0,100000000000.0,3.4,-82.17357,2018-01-07T00:00:00.000Z,0.00268621,22453.08311,2014-05-22T00:00:00.000Z,None,2023-08-15T20:49:11.439Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,terra-luna,lunc,Terra Luna Classic,https://assets.coingecko.com/coins/images/8284...,7.098e-05,412407630,96,484895634.0,39656605.0,7.949e-05,...,6838817824102.49,nan,119.18,-99.99994,2022-04-05T12:24:58.854Z,9.999669999999999e-07,6989.27766,2022-05-13T02:34:40.340Z,None,2023-08-15T20:49:19.019Z
96,sui,sui,Sui,https://assets.coingecko.com/coins/images/2637...,0.561632,406106156,97,5615699510.0,60441115.0,0.601919,...,10000000000.0,10000000000.0,2.16,-74.03938,2023-05-03T12:00:26.430Z,0.550692,1.9449100000000001,2023-08-15T19:34:59.901Z,None,2023-08-15T20:49:00.469Z
97,gmx,gmx,GMX,https://assets.coingecko.com/coins/images/1832...,45.15,404186632,98,598324527.0,13507932.0,47.63,...,8951069.00287494,13250000.0,91.07,-50.38713,2023-04-18T10:00:25.680Z,11.53,291.74364,2022-06-15T09:30:22.146Z,None,2023-08-15T20:49:09.041Z
98,huobi-token,ht,Huobi,https://assets.coingecko.com/coins/images/2822...,2.55,403615196,99,509057350.0,9123452.0,2.59,...,201050000.0,201050000.0,39.66,-93.61587,2021-05-12T14:42:21.586Z,0.3138,706.88138,2023-03-10T05:05:00+08:00,"{'times': 1.5707538705892552, 'currency': 'usd...",2023-08-15T20:49:11.601Z
